# A seguinte uma classificação de tópicos utilizando NLP e machine learning para o teste técnico da NeuralMED

# Importação de bibliotecas

In [ ]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn import metrics
from nltk.tokenize import TweetTokenizer

In [ ]:
df = pd.read_csv('TweetsNeutralHash.csv', encoding='utf-8')

# Tokenizador de texto específico para twitter

In [ ]:
tweet_tokenizer = TweetTokenizer()

# Funções de préprocessamento dos dados

In [ ]:
def removestopwords(texto):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in texto if not i in stopwords]
    return (palavras)

In [ ]:
def clean_data(texto):
    texto = re.sub(r'http\S+', '', texto).lower().replace('.', '').replace(';', '').replace(',', '')
    return texto

In [ ]:
def processar_texto(texto):
    texto = clean_data(texto)
    texto_tokenizado = tweet_tokenizer.tokenize(texto)
    texto_tokenizado = removestopwords(texto_tokenizado)
    return (" ".join(texto_tokenizado))

In [ ]:
tweets = df['tweet_text']

In [ ]:
tweets.head()

In [ ]:
tweets = [processar_texto(tweet) for tweet in tweets]

In [ ]:
assunto = df['query_used'].replace('#','')
assunto

# Vetorizando os textos para alimentar o modelo de machine learning

In [61]:
matriz = CountVectorizer(max_features=1000)
vetores = matriz.fit_transform(tweets).toarray()

In [64]:
from sklearn.model_selection import train_test_split
vetores_train, vetores_test, topicos_train, topicos_test = train_test_split(vetores, assunto) 

# Criando o modelo de machine learning (Naive Bayes)

In [65]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(vetores_train, topicos_train)

GaussianNB()

In [66]:
topicos_pred = classifier.predict(vetores_test)

# Métricas de precisão

In [67]:
from sklearn.metrics import classification_report
print(classification_report(topicos_test, topicos_pred))

               precision    recall  f1-score   support

 #curiosidade       0.10      0.68      0.18       130
        #fato       0.92      0.57      0.70      1392
     #noticia       0.65      0.68      0.66       437
    #novidade       0.47      0.82      0.59       330
#oportunidade       0.80      0.90      0.85       715
    #trabalho       0.81      0.33      0.47       928

     accuracy                           0.61      3932
    macro avg       0.62      0.66      0.58      3932
 weighted avg       0.78      0.61      0.64      3932

